# GIRA - Extrair disponibilidade bicicletas

Manuel Banza - February, 2022

In [48]:
import requests
import pandas as pd
import datetime
import time

In [68]:
url = 'http://coiapp.cm-lisboa.pt/api/opendata/public/download/53616c7465645f5f62a975d9194dd0ee7a0d7ffb4f36b0c0b5161a94daf4ed0e.csv?format=csv'
df = pd.read_csv(url, encoding='utf8')
print(len(df))

# Passar para date e criar coluna datetime
df['add_data'] = pd.to_datetime(df['add_data'], format='%Y-%m-%d')
df['datetime'] = df['add_data'].astype(str) + ' ' + df['add_hora'].astype(str)
df['datetime'] = pd.to_datetime(df['datetime'])

# Limpar coluna com coordenadas
df['position'] = df['position'].astype(str)
df['position'] = df['position'].str[17:]
df['position'] = df['position'].str[:20]
df['position'] = df['position'].str.replace('],', '')

# cirar latitude e longitude
df[['longitude', 'latitude']] = df['position'].str.split(', ', expand=True)

# Eliminar coluna position
df = df.drop(columns=['position'])

df.head(3)

120


<ipython-input-68-c7637f8937c5>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['position'] = df['position'].str.replace('],', '')


,desigcomercial,numbicicletas,numdocasvacias,add_data,add_hora,datetime,longitude,latitude
0,475 - Av. Professor Gama Pinto / Cantina Velha,6,35,2022-02-08,18:56:16.,2022-02-08 18:56:16,-9.15907,38.75115
1,476 - Av. Professor Gama Pinto / Reitoria,16,7,2022-02-08,18:56:16.,2022-02-08 18:56:16,-9.15885,38.75231
2,460 - Av. Rio de Janeiro / Parque de Jogos 1º ...,15,0,2022-02-08,18:56:16.,2022-02-08 18:56:16,-9.13749,38.75176


In [8]:
# Passar para date e criar coluna datetime
print(df.datetime.min())
print(df.datetime.max())
df.head(3)

2019-07-01 16:35:49
2022-02-08 18:56:22


,desigcomercial,numbicicletas,numdocasvacias,position,add_data,add_hora,datetime
0,Avenida das Gaivotas - Rua dos Corvos,18,2,"{""coordinates"": [-9.180755, 38.769083], ""type""...",2020-09-24,10:52:51.,2020-09-24 10:52:51
1,220 - Rua do Comercio,9,11,"{""coordinates"": [-9.137125, 38.70878], ""type"":...",2022-02-08,18:56:13.,2022-02-08 18:56:13
2,463 - Campo Grande / Av. da Igreja,2,8,"{""coordinates"": [-9.14951, 38.75243], ""type"": ...",2022-02-08,18:56:14.,2022-02-08 18:56:14


In [69]:
from datetime import datetime, timedelta

yesterday = datetime.now() - timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')

In [70]:
print(len(df))
df = df.loc[df['add_data']>=yesterday]
print(len(df))

120
113


In [ ]:
# Get today date now to file name when export to csv or excel with encoding utf8
from datetime import datetime
df.to_csv(datetime.now().strftime('data_sources/data_transformed/gira_disponibilidade-%Y-%m-%d-%H-%M-%S.csv'), encoding='utf8', index=False)